In [1]:
# ERROR:


import re

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from ast import literal_eval
from collections import OrderedDict

%matplotlib inline

In [2]:
prep_labels = [
    'Shape', 
    'First Order', 
    'GLCM', 
    'GLRLM', 
    'GLSZM', 
    'GLDM', 
    'NGTDM', 
    'PET parameter',
    'Clinical'
]
colours = OrderedDict(
    [
        ('Shape', sns.color_palette()[0]),
        ('Clinical', sns.color_palette()[1]),
        ('First Order', sns.color_palette()[2]),
        ('GLCM', sns.color_palette()[3]),
        ('GLRLM', sns.color_palette()[8]),
        ('GLSZM', sns.color_palette()[5]),
        ('GLDM', sns.color_palette()[6]),
        ('NGTDM', sns.color_palette()[7]),
        ('PET parameter', sns.color_palette()[4])
    ]
)

In [7]:
X = pd.read_csv('./../../../data_source/to_analysis/no_filter_concat.csv', index_col=0)
features = list(X.columns)
features[:6], len(features)

(['original_shape_Elongation',
  'original_shape_Flatness',
  'original_shape_LeastAxis',
  'original_shape_MajorAxis',
  'original_shape_Maximum2DDiameterColumn',
  'original_shape_Maximum2DDiameterRow'],
 610)

In [4]:
#results = pd.read_csv('./../../../data_source/results/baseline_nofilter_sffs_40vals_10reps_dfs.csv', index_col=0)
results = pd.read_csv('./../../../data_source/results/with_pet_outliers_univariate_sffs_nofilter_dfs.csv', index_col=0)
results.head()

,exp_id,experiment_id,test_score,train_score,test_score_variance,train_score_variance,feature_votes,exp_duration
0,684,StudentTTestSelection_DTreeEstimator,0.629304,0.657456,0.022481,0.000472,[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 ...,0 days 00:10:12.873585000
1,559,StudentTTestSelection_DTreeEstimator,0.528846,0.590600,0.010256,0.000747,[10 10 10 10 10 10 10 7 8 10 7 9 8 3 3 ...,0 days 00:09:16.610746000
2,629,StudentTTestSelection_DTreeEstimator,0.512637,0.651458,0.006317,0.000645,[9 5 5 8 7 4 8 7 6 5 4 8 7 7 6 6 3 6 4 4 7 4 4...,0 days 00:11:01.242483000
3,192,StudentTTestSelection_DTreeEstimator,0.562912,0.696808,0.012518,0.000340,[ 6 8 6 4 5 8 7 6 7 6 5 5 8 7 7 ...,0 days 00:10:16.402706000
4,835,StudentTTestSelection_DTreeEstimator,0.594963,0.662974,0.018951,0.000213,[10 10 10 10 9 10 10 10 10 10 10 9 10 10 10 ...,0 days 00:11:40.178124000


In [5]:
votes = results['feature_votes']
votes.head()

0    [10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 ...
1    [10 10 10 10 10 10 10  7  8 10  7  9  8  3  3 ...
2    [9 5 5 8 7 4 8 7 6 5 4 8 7 7 6 6 3 6 4 4 7 4 4...
3    [ 6  8  6  4  5  8  7  6  7  6  5  5  8  7  7 ...
4    [10 10 10 10  9 10 10 10 10 10 10  9 10 10 10 ...
Name: feature_votes, dtype: object

In [29]:
len(list(votes.values[0].replace('\n', '').replace('[', '').replace(']', '').replace(' ', '')))

627

In [11]:
len(list(votes[0].replace(' ', '').replace('\n', ''))[1:-1])

627

In [6]:
# Goal:
# * Sort votes and collect corresponding labels.

global_votes = np.zeros(X.shape[1], dtype=int)
for str_array in votes:
    _votes = list(str_array.replace(' ', '').replace('\n', ''))
    votes_arr = np.array(list(map(int, list(_votes[1:-1]))), dtype=int)
    global_votes += votes_arr

idx = np.argsort(global_votes)[::-1]
labels = np.array(features)[idx]
sorted_votes = np.trim_zeros(global_votes[idx], trim='b')
labels = labels[:len(sorted_votes)]


data = pd.DataFrame(
    {
        'votes': sorted_votes,
        'labels': labels,
        
    }
)
data.head()

ValueError: operands could not be broadcast together with shapes (610,) (627,) (610,) 

In [ ]:
# Sanity check.
max(global_votes), sorted_votes[0]

In [ ]:
len(global_votes), len(sorted_votes)

In [ ]:
def prep_feature_labels(labels):
    prep_labels = []
    for label in labels:
        comps = label.split('_')

        if len(comps) == 1:
            prep_labels.append(label.title())
        elif len(comps) == 2:
            new_label = '{}: {}'.format(comps[0], comps[1])
            prep_labels.append(new_label)
        elif len(comps) == 3:
            filter_type, feature_type, name = comps
            if len(name) > 15:
                #name_comps = re.findall('[A-Z][^A-Z]*', name)
                #name = ('-\n').join((('').join(name_comps[:2]), ('').join(name_comps[2:])))
                new_label = '{}'.format(name)
            else:
                new_label = '{}'.format(name)
            prep_labels.append(new_label)
        elif len(comps) == 4:
            image_type, filter_type, feature_type, name = comps
            if len(name) > 15:
                #name_comps = re.findall('[A-Z][^A-Z]*', name)
                #name = ('-\n').join((('').join(name_comps[:2]), ('').join(name_comps[2:])))
                new_label = '\n{}'.format(name)
            else:
                new_label = '{}'.format(name)
            prep_labels.append(new_label)
            
    return prep_labels

In [ ]:
def feature_labels(labels):    
    
    keys = []
    for label in labels:
        if 'shape' in label:
            keys.append('Shape')
        elif 'firstorder' in label:
            keys.append('First Order')
        elif 'glcm' in label:
            keys.append('GLCM')
        elif 'glrlm' in label:
            keys.append('GLRLM')
        elif 'glszm' in label:
            keys.append('GLSZM')
        elif 'gldm' in label:
            keys.append('GLDM')
        elif 'ngtdm' in label:
            keys.append('NGTDM')
        elif 'PETparam' in label:
            keys.append('PET parameter')
        else:
            keys.append('Clinical')

    return keys

In [ ]:
keys = feature_labels(labels)
handles = [
    mpatches.Patch(color=colours[key], label=key) for key in prep_labels
]
len(keys), len(handles), len(colours)

In [ ]:
# TODO: Include 15 least interesting features.

fig, axes = plt.subplots(nrows=2, figsize=(15, 9))
plt.tick_params(axis='both', labelsize=12)

sns.barplot(
    x=np.arange(np.size(sorted_votes)), 
    y=sorted_votes, 
    ax=axes[0], 
    hue=keys,
    palette=colours,
)
feature_id = range(1, sorted_votes.size, 5)
axes[0].set_title('Frequency of selected features', fontsize=15)
axes[0].set_xticks(feature_id)
axes[0].set_ylabel('Counts', fontsize=16)
axes[0].set_xlabel('Feature Number Indicator', fontsize=16)
axes[0].legend(
    handles=handles,
    title='Feature Categories:', 
    title_fontsize=15,
    loc='upper center', 
    bbox_to_anchor=(0.5, -0.25), 
    fancybox=True, 
    shadow=True, 
    fontsize=14,
    ncol=4,
    labelspacing=0.25
)

n = 15
keys = feature_labels(labels[:n])
sns.barplot(
    x=np.arange(np.size(sorted_votes[:n])), 
    y=sorted_votes[:n], 
    ax=axes[1], 
    hue=keys,
    palette=colours,
)
fnames = prep_feature_labels(labels[:n])
#axes[1].legend().set_visible(False)
axes[1].set_title('The {} most frequently selected features'.format(n), fontsize=15)
axes[1].set_xticklabels(fnames, rotation=45, ha='right', va='center_baseline')
axes[1].set_ylabel('Counts', fontsize=14)

plt.subplots_adjust(hspace=1)

plt.savefig(
    './../../../figures/biomarkers_200evals_40reps.png',
    bbox_inches='tight',
    transparent=True,
    dpi=600, 
)

In [ ]:
cats = ['shape',
        'firstorder',
        'glcm',
        'glrlm',
        'glszm',
        'gldm',
        'ngtdm']
res = {}
for cat in cats:
    p = 0
    c = 0
    for label in labels:
        if cat in label:
            c = c + 1
    res[cat] = c
            
res

In [ ]:
sorted_votes

In [ ]:
# NOTE: Num clinical features.
len(labels) - sum(res.values())

In [ ]:
len(list(X.filter(regex='shape').columns))

In [ ]:
len(list(X.filter(regex='CT').columns))

In [ ]:
len(list(X.filter(regex='PET').columns))

In [ ]:
nogos = list(X.filter(regex='shape').columns) 
nogos = nogos + list(X.filter(regex='CT').columns) 
nogos = nogos + list(X.filter(regex='PET').columns)

cols = {}
for num, col in enumerate(X.columns):
    if col not in nogos:
        cols[col] = sorted_votes[num]
        print(col, ':', sorted_votes[num])
        
# Pasienten er mann/kvinne er relativt viktig. Det er flere menn enn kvinner
# i datasettet. Kan være knyttet til HPV variabel fordi ikke utypisk at menn får HPV?.

# 

In [ ]:
x_coords = np.arange(len(cols.values()))
clinical_labels = prep_feature_labels(cols.keys())

plt.figure(figsize=(16, 7))
plt.title('Selected Clinical Features', fontsize=16)
sns.barplot(x=x_coords, y=list(cols.values()))
plt.xticks(x_coords, clinical_labels, rotation=45, ha='right', fontsize=14)
plt.ylabel('Number of Votes', fontsize=16)
plt.ylim([0, 250])
plt.tight_layout()
plt.savefig(
    './../../../figures/clinical_feature_biomarker.png',
    bbox_inches='tight',
    transparent=True,
    dpi=600, 
)

In [ ]:
# Group by encoded features.
df_clinical = pd.DataFrame.from_dict(cols, orient='index', columns=['Votes'])
df_clinical.head()

In [ ]:
grouper = []
for num, label in enumerate(df_clinical.index):
    if 'ICD-10' in label:
        grouper.append('IDC-10')
    elif 'Stage' in label:
        grouper.append('Stage')
    elif 'Histology' in label:
        grouper.append('Histology')
    elif 'HPV' in label:
        grouper.append('HPV')
    elif 'ECOG' in label:
        grouper.append('ECOG')
    elif 'Charlson' in label:
        grouper.append('Charlson')
    elif 'Cisplatin' in label:
        grouper.append('Cisplatin')
    else:
        grouper.append(label)

df_clinical['groups'] = grouper

df_clinical.head()

In [ ]:
# Missing: 
df_clinical_grps = df_clinical.groupby('groups').sum()

plt.figure(figsize=(15, 8))
sns.set(font_scale=1.5)
sns.barplot(y='Votes', x=df_clinical_grps.index, data=df_clinical_grps)
plt.xlabel('')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(
    './../../../figures/grouped_clinical_feature_biomarker.png',
    bbox_inches='tight',
    transparent=True,
    dpi=600, 
)

In [ ]:
idx = int(np.squeeze(np.where(results['test_score'] == results['test_score'].max())))
best_run = results.iloc[idx, :]
best_run_votes = best_run['feature_votes']
_votes = list(best_run_votes.replace(' ', '').replace('\n', ''))
votes_arr = np.array(list(map(int, list(_votes[1:-1]))), dtype=int)

idx = np.argsort(votes_arr)[::-1]
labels = np.array(features)[idx]
sorted_votes = np.trim_zeros(votes_arr[idx], trim='b')
labels = prep_feature_labels(labels[:len(sorted_votes)])

y_coords = np.arange(np.size(labels))

plt.figure(figsize=(8, 15))
sns.set(font_scale=1.5)
sns.barplot(y=labels, x=sorted_votes, )
plt.xlabel('Votes')
plt.yticks(y_coords, labels, va='bottom', fontsize=14)
plt.tight_layout()

plt.savefig(
    './../../../figures/best_run_features.png',
    bbox_inches='tight',
    transparent=True,
    dpi=600, 
)

In [ ]:
# Group by encoded features.
df_clinical = pd.DataFrame(sorted_votes, columns=['Votes'])
df_clinical.index = labels
df_clinical.head()

In [ ]:
grouper = []
for num, label in enumerate(df_clinical.index):
    if 'ICD-10' in label:
        grouper.append('IDC-10')
    elif 'Stage' in label:
        grouper.append('Stage')
    elif 'Histology' in label:
        grouper.append('Histology')
    elif 'HPV' in label:
        grouper.append('HPV')
    elif 'ECOG' in label:
        grouper.append('ECOG')
    elif 'Charlson' in label:
        grouper.append('Charlson')
    elif 'Cisplatin' in label:
        grouper.append('Cisplatin')
    else:
        grouper.append(label)

df_clinical['groups'] = grouper

df_clinical.head()

In [ ]:
# Missing: 
df_clinical_grps = df_clinical.groupby('groups').sum()

plt.figure(figsize=(10, 15))
sns.set(font_scale=1.5)
sns.barplot(x='Votes', y=df_clinical_grps.index, data=df_clinical_grps)
plt.xlabel('Votes')
plt.ylabel('')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(
    './../../../figures/grouped_best_run_features.png',
    bbox_inches='tight',
    transparent=True,
    dpi=600, 
)